In [15]:
 #imports
import torch
import torch.nn as nn # All Neural Network models
import torch.optim as optim # All Optimization
from torch.utils.data import DataLoader # Easier Dataset Managment 
from torch.utils.tensorboard import SummaryWriter
import torchvision 
import torchvision.transforms as transforms # Transformations we can do
import torchvision.datasets as datasets # to import the needed dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification
import os

In [16]:
# Transformation 
# Define the standard ImageNet transforms
transform = transforms.Compose([
    transforms.Resize(256),              # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),          # Crop the image to 224x224 pixels at the center
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # Normalize pixel values to match the ImageNet distribution
                         std=[0.229, 0.224, 0.225]),
])

# Load Training & Test DataSets
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
# Set Device
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
device = torch.device("cuda")

#HyperParameters 
batch_size = 64
epochs = 5
learning_rate = 0.001

#Data Loader
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Load a pre-trained model and feature extractor
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-base')
# Adjust the model for CIFAR-100
model.classifier = torch.nn.Linear(model.classifier.in_features, 100)
_ = model.to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
torch.cuda.empty_cache()

# Initialize TensorBoard
# writer = SummaryWriter('Fine-Tuning_CIFAR100')

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Initialize TensorBoard
writer = SummaryWriter('runs/Fine-Tuning')
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training The Model
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
  
    for i, data in enumerate(trainloader, 0):
        scheduler.step() 
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = criterion(
            outputs, labels
        )
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # if i % (batch_size * 2) == ((batch_size * 2) - 1):
        #     print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
        #     running_loss = 0.0
    epoch_loss = running_loss / len(trainloader)
    
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(
                outputs.logits, 1
            )
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_accuracy = 100 * correct / total
        # Log loss and accuracy for the epoch to TensorBoard
        writer.add_scalar('Accuracy/test', epoch_accuracy, epoch)
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        
        print(f'Accuracy : {epoch_accuracy} % ' + ' loss: %.3f' % (running_loss / 100))
    # print('loss: %.3f' % (running_loss / 100))
print('Finished Training')
writer.close()

Accuracy : 1.25 %  loss: 38.388
Accuracy : 1.25 %  loss: 38.388
Accuracy : 1.25 %  loss: 38.387
Accuracy : 1.25 %  loss: 38.387
Accuracy : 1.25 %  loss: 38.386
Finished Training
